# Running instructions

## Change working directory

In [ ]:
cd ~/workspace/genome_data/annotations/e_coli/gene_sequences/fhu_system
source ~/workspace/alfred-data-analysis/.venv_gwas/bin/activate

## Obtain variants from the multiple sequence alignment files

In [ ]:
snp-sites -v -o variants.vcf aligned.fna

## Obtain consensus sequence from the multiple sequence alignment files

In [51]:
import os
from pathlib import Path

from Bio import AlignIO
from Bio.Align import AlignInfo


dataDir = Path(os.environ['GENOMICS_DATA_BASE'], 'annotations', 'e_coli', 'gene_sequences', 'fhu_system')

alignment = AlignIO.read(Path(dataDir, "aligned.fna"), "fasta")

summary_align = AlignInfo.SummaryInfo(alignment)

consensus = summary_align.dumb_consensus(threshold=0.40, ambiguous='N')

In [52]:
print(consensus.count('N'))

0


In [53]:
print(consensus)

ATGGCACCTTCCAAAACTGCTCAGCCAAAACACTCACTGCGTAAAATCGCAGTTGTAGTAGCCACAGCGGTTAGCGGCATGTCTGTTTATGCACAGGCAGCGGTTGAACCGAAAGAAGACACTATCACCGTTACCGCTGCACCTGCGCCGCAAGAAAGCGCATGGGGGCCGGCTGCAACTATTGCGGCGCGACAGTCAGCTACCGGCACTAAAACCGATACGCCGATTCAAAAAGTACCTCAATCTATTTCTGTCGTCACCGCTGAAGAGATGGCGCTGCATCAGCCTAAGTCGGTGAAAGAAGCGCTTAGCTATACCCCTGGCGTTGCTGTGGGAACCCGTGGCGCATCTAATACTTATGATTACCTGATCATCCGCGGATTTGCGGCAGACGGCCAAAGCCAGAATAACTATCTGAATGGCCTGAAGTTGCAGGGCAACTTCTATAACGATGCGGTCATCGACCCGTATATGCTGGAACGTGCTGAAATTATGCGTGGCCCGGTTTCCGTGCTTTACGGTAAAAGCAGTCCTGGCGGTCTGTTGAATATGGTCAGCAAGCGTCCGACCACTGAACCGTTGAAAGAAGTTCAGTTTAAGGCCGGGACTGACAGCCTGTTCCAGACCGGTTTTGACTTCAGCGATGCGCTGGATGATGACGGCGTTTATTCTTATCGCCTGACCGGTCTTGCGCGTTCTGCCAATGCCCAGCAGAAAGGGTCAGAAGAGCAGCGTTATGCTATTGCACCGGCGTTCACCTGGCGTCCGGATGATAAAACCAATTTCACCTTCCTTTCTTACTTCCAGAACGAGCCGGAAACTGGTTATTACGGCTGGTTGCCGAAAGAGGGGACCGTTGAGCCGCTGCCAAACGGTAAGCGTCTGCCGACAGACTTTAACGAAGGGGCGAAGAACAACACCTATTCTCGTAACGAGAAGATGATTGGTTATAGCTTCGACCACGAATTTAACGACACCTTTACTGTGCGTCAGAACCTGC

In [54]:
print(len(consensus))

5980


### Translate the nucleotide sequence to amino acid sequence

In [55]:
from Bio.Seq import Seq


dna_seq = Seq(consensus)
protein_seq = dna_seq.translate()
protein_seq

/home/vmadmin/workspace/alfred-data-analysis/.venv/lib/python3.10/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('MAPSKTAQPKHSLRKIAVVVATAVSGMSVYAQAAVEPKEDTITVTAAPAPQESA...EPL')

In [57]:
str(protein_seq)

'MAPSKTAQPKHSLRKIAVVVATAVSGMSVYAQAAVEPKEDTITVTAAPAPQESAWGPAATIAARQSATGTKTDTPIQKVPQSISVVTAEEMALHQPKSVKEALSYTPGVAVGTRGASNTYDYLIIRGFAADGQSQNNYLNGLKLQGNFYNDAVIDPYMLERAEIMRGPVSVLYGKSSPGGLLNMVSKRPTTEPLKEVQFKAGTDSLFQTGFDFSDALDDDGVYSYRLTGLARSANAQQKGSEEQRYAIAPAFTWRPDDKTNFTFLSYFQNEPETGYYGWLPKEGTVEPLPNGKRLPTDFNEGAKNNTYSRNEKMIGYSFDHEFNDTFTVRQNLRFAQNKVSQKSVYGYGMCSDPLYT*NQEALKASPCASIPQSQWGHTLTRQYVIDNEKLENFSVDTQLQSKFATGSVDHTLLTGVDFMRMRNDIDSWFGYAGSVAPSDIYNLDRSDFDFGANHPNPSGPYRVLLKQKQTGLYVQDQAQWDKVLVTLGGRYDWADQSSFNRDYGNKSERDDKEFTWRGGVNYLFDNGVTPYFSYSESFEPASQTDANGDLFAPSKGKQYEVGVKYVPEDRPIVVTGALYQLTKTNNLMADPNGSLFSVEGGEIRARGVELEAKAALSASVNVVGSYTYTDAEYTTDTTYKGNTPAQVPKHMASLWADYTFFDGPLSGLTLGTGGRYTGSSYGDPANSFKVGSYTVVDALVRYDLARVGMAGSNVALHVNNLFDREYVASCFNTYGCFWGAERQVVATATFRF*FLFLGHGFPCPFHKLAVMQEYTNHSDTTFALRNISFRVPGRTLLHPLSLTFPAGKVTGLIGHNGSGKSTLLKMLGRHQPPSEGEILLDAQPLESWSSKAFARKVAYLPQQLPPAEGMTVRELVAIGRYPWHGALGRFGAADREKVEEAISLVGLKPLAHRLVDSLSGGERQRAWIAMLVAQDSRCLLLDEPTSALDIAHQVDVLALVHRLSQERGLTVIAVLHDINMAARYCDYLVALRGGEMIA

## Patristic distances

### Extract distance from phylogeny

In [ ]:
python ~/workspace/pyseer/scripts/phylogeny_distance.py tree.nwk  > phylogeny_dists.tsv

### Perform GWAS

In [ ]:
pyseer --phenotypes /home/vmadmin/workspace/ehr_data/data/full_cohort/tube_id_mortality.pheno --vcf variants.vcf --distances phylogeny_dists.tsv --lineage --max-dimensions 6 --min-af 0.06 --max-af 0.94 > mortality_SNPs.txt

### Format output

In [ ]:
cat <(echo "#CHR SNP BP minLOG10(P) log10(p) r^2") <(paste <(sed '1d' mortality_SNPs.txt | cut -d "_" -f 2) <(sed '1d' mortality_SNPs.txt | cut -f 4) | awk '{p = -log($2)/log(10); print "1",".",$1,p,p,"0"}' ) | tr ' ' '\t' > mortality_snps.plot

## Distance from root

### Extract distance from phylogeny

In [ ]:
python ~/workspace/pyseer/scripts/phylogeny_distance.py --lmm tree.nwk  > phylogeny_K.tsv

### Perform GWAS

In [ ]:
python ~/workspace/pyseer/pyseer-runner.py --lmm --phenotypes ~/workspace/ehr_data/data/full_cohort/tube_id_mortality.pheno --vcf variants.vcf --similarity phylogeny_K.tsv --phenotype-column death_30_day --output-patterns mortality_SNP_patterns.txt > mortality_SNPs_lmm.txt

### Analyse output

#### Count the number of patterns to control for multiple testing

In [ ]:
python ~/workspace/pyseer/scripts/count_patterns.py mortality_SNP_patterns.txt

Output:

Patterns:	98

Threshold:	5.10E-04

In [ ]:
library(data.table)


gono_gwas <- fread('~/workspace/genome_data/annotations/e_coli/gene_sequences/fhu_system/mortality_SNPs_lmm.txt', data.table = FALSE)
head(gono_gwas)

,variant,af,filter-pvalue,lrt-pvalue,beta,beta-std-err,variant_h2,notes
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1_6_A_G,0.3960,0.2840,0.2850,0.02370,0.0221,0.03800,
2,1_8_C_G,0.3960,0.2840,0.2850,0.02370,0.0221,0.03800,
3,1_33_C_T,0.0202,0.1700,0.1700,-0.10600,0.0770,0.04880,bad-chisq
4,1_72_T_A,0.0391,0.9000,0.9000,-0.00704,0.0559,0.00448,
5,1_147_G_A,0.0303,0.0909,0.0911,-0.10700,0.0631,0.06010,bad-chisq
6,1_210_T_C,0.0290,0.0981,0.0984,-0.10700,0.0644,0.05880,bad-chisq


In [ ]:
gono_gwas <- gono_gwas[order(gono_gwas$`lrt-pvalue`),]
gono_gwas <- gono_gwas[!grepl("bad-chisq", gono_gwas$notes),]

# threshold form running count_patterns in pyseer
sig_threshold <- 0.05/(98)
sig_threshold

[1] 0.0005102041

In [ ]:
sum(gono_gwas$`lrt-pvalue`<sig_threshold)
sig_hits <- gono_gwas[gono_gwas$`lrt-pvalue`<sig_threshold,]
sig_hits

[1] 13

,variant,af,filter-pvalue,lrt-pvalue,beta,beta-std-err,variant_h2,notes
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
560,1_4841_C_T,0.0227,1.57e-06,1.36e-06,0.349,0.0717,0.171,
537,1_4600_C_T,0.0240,4.26e-06,3.80e-06,0.325,0.0699,0.163,
539,1_4615_A_G,0.0253,1.04e-05,9.50e-06,0.304,0.0682,0.157,
562,1_4853_T_C,0.0543,1.08e-05,9.86e-06,0.210,0.0472,0.156,
563,1_4855_A_G,0.0568,2.64e-05,2.45e-05,0.197,0.0463,0.149,
564,1_4858_T_C,0.0568,2.64e-05,2.45e-05,0.197,0.0463,0.149,
565,1_4861_G_C,0.0568,2.64e-05,2.45e-05,0.197,0.0463,0.149,
553,1_4814_T_C,0.0581,4.00e-05,3.74e-05,0.190,0.0458,0.146,
554,1_4817_T_C,0.0593,5.94e-05,5.59e-05,0.184,0.0454,0.143,


## Pairwise distance matrix produced using mash

### Create the mash sketches

In [ ]:
mash sketch -s 10000 -o mash_sketch /home/vmadmin/workspace/genome_data/fasta/ECOLI/*fasta

### Calculate distances between all pairs of samples

In [ ]:
mash dist mash_sketch.msh mash_sketch.msh| square_mash > mash.tsv

In [ ]:
python ~/workspace/pyseer/scree_plot.py mash.tsv

In [ ]:
sed -i 's/_short//g' mash.tsv

### Perform GWAS

In [ ]:
python ~/workspace/pyseer/pyseer-runner.py --phenotypes ~/workspace/ehr_data/data/full_cohort/tube_id_mortality.pheno --vcf variants.vcf --distances mash.tsv --phenotype-column death_30_day --output-patterns mortality_SNP_patterns_mash.txt --max-dimensions 6 --min-af 0.085 --max-af 0.915 > mortality_SNPs_mash.txt